In [5]:
import dash
from dash import html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
from dash.exceptions import PreventUpdate

# Load your data (replace 'df_ca2_dataviz.csv' with your actual data file)
df = pd.read_csv('df_ca2_dataviz.csv')

# GeoJSON for Ireland counties (you may need to replace this with a valid GeoJSON file for Ireland)
ireland_geojson = {
    "type": "FeatureCollection",
    "features": [
        {"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[-9.753111, 51.588663], [-9.385891, 51.375365], [-8.922242, 51.624837], [-8.341016, 51.80129], [-8.549227, 52.194792], [-9.238723, 52.335385], [-9.472834, 52.265139], [-9.824383, 51.823769], [-9.753111, 51.588663]]]}}
    ]
}

# Create Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div(children=[
    html.H1(children='Your Dashboard Title'),

    # First Row
    html.Div(children=[
        # Panel 1: Text
        dcc.Markdown('''
            # Panel 1
            Your text here.
        '''),
    ]),

    # Second Row
    html.Div(children=[
        # Panel 2: Line Chart for Names by Year

        # Input for the name
        dcc.Input(
            id='name-input',
            type='text',
            placeholder='Enter Name',
            value='',
            style={'margin-bottom': '10px'}
        ),

        # Line Chart based on the entered name
        dcc.Graph(
            id='line-chart',
        ),
    ]),

    # Third Row
    html.Div(children=[
        # Panel 3: Choropleth Map for Count by State (USA)
        dcc.Graph(
            id='choropleth-map-usa',
        ),
    ]),

    # Fourth Row
    html.Div(children=[
        # Panel 4: Choropleth Map for Count in Ireland
        dcc.Graph(
            id='choropleth-map-ireland',
        ),
    ]),
])

# Callbacks
@app.callback(
    Output('line-chart', 'figure'),
    [Input('name-input', 'value')]
)
def update_line_chart(selected_name):
    filtered_df = df[df['Name'] == selected_name]
    fig = px.line(filtered_df, x='Year', y='Count', title=f'Count Over Years for {selected_name}')
    return fig

@app.callback(
    Output('choropleth-map-usa', 'figure'),
    [Input('name-input', 'value')]
)
def update_choropleth_map_usa(selected_name):
    if not selected_name:
        raise PreventUpdate

    # Aggregate count for each state and USA across all years
    aggregated_data_usa = df[(df['Name'] == selected_name) & (df['Country'] == 'USA')].groupby('State')['Count'].sum().reset_index()

    # Create a choropleth map for the USA
    fig_usa = px.choropleth(aggregated_data_usa, 
                            locations='State', 
                            locationmode='USA-states', 
                            color='Count',
                            title=f'Choropleth Map for {selected_name} by State (USA)',
                            color_continuous_scale='viridis',
                            scope='usa')

    return fig_usa

@app.callback(
    Output('choropleth-map-ireland', 'figure'),
    [Input('name-input', 'value')]
)
def update_choropleth_map_ireland(selected_name):
    if not selected_name:
        raise PreventUpdate

    # Aggregate total count for Ireland across all years
    aggregated_data_ireland = df[(df['Name'] == selected_name) & (df['Country'] == 'Ireland')]['Count'].sum()

    # Create a choropleth map for Ireland
    fig_ireland = px.choropleth_mapbox(geojson=ireland_geojson,
                                       locations=["Ireland"],
                                       color=[aggregated_data_ireland],
                                       color_continuous_scale='viridis',
                                       title=f'Total Count for {selected_name} in Ireland (All Years)',
                                       mapbox_style="carto-positron",
                                       center={"lat": 53.4129, "lon": -8.2439},
                                       zoom=6)

    return fig_ireland

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
